# Using asymmetric IRFs

While Gammapy does not have inbuilt classes for supporting asymmetric IRFs (except for `Background3D`), custom classes can be created. For this to work correctly with the `MapDatasetMaker`, only variations with `fov_lon` and `fov_lat` can be allowed. 

Analytic models for the PSF, or anisotropic PSFs (ie, asymmetry around the source position) is not correctly supported during the data reduction

In [154]:
import numpy as np
from gammapy.irf import *
import astropy.units as u
import scipy.special
from gammapy.maps import MapAxis, MapAxes
from gammapy.irf.io import IRF_DL3_HDU_SPECIFICATION, COMMON_IRF_HEADERS

In [83]:
irf_filename = (
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)
irfs = load_irf_dict_from_file(irf_filename)

Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [168]:
coords = {}
coords["offset"] = [[0.16222949, 0.16222949, 1.2423175 , 1.83029858],[0.16222949, 0.16222949, 1.2423175 , 1.83029858]
                   ]*u.deg 
coords["energy_true"] = [1,10]*u.TeV

In [169]:
irfs["aeff"].evaluate(**coords)

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (2,) and arg 1 with shape (2, 4).

## Effective Area

In [84]:
class EffectiveArea3D(IRF):
    tag = "aeff_3d"
    required_axes = ["energy_true", "fov_lon", "fov_lat"]
    default_unit = u.m**2

In [85]:

energy_axis = MapAxis.from_energy_edges([0.1, 0.3, 1.0, 3.0, 10.0]*u.TeV, name="energy_true")

nbin=7
fov_lon_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lon")
fov_lat_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lat")

data = np.ones((4, 3, 3))
for i in range(1,4):
    data[i] = data[i-1]*1.5

aeff_3d = EffectiveArea3D([energy_axis, fov_lon_axis, fov_lat_axis], data=data, unit=u.m**2)
print(aeff_3d)

EffectiveArea3D
---------------

  axes  : ['energy_true', 'fov_lon', 'fov_lat']
  shape : (4, 3, 3)
  ndim  : 3
  unit  : m2
  dtype : float64



In [86]:
aeff_3d.evaluate(fov_lon=[-0.5, 0.8] * u.deg,
        fov_lat=[-0.5, 1.0] * u.deg,
        energy_true=[0.2, 8.0] * u.TeV,)

<Quantity [1.06246937, 3.74519106] m2>

### Serialisation

In [155]:
IRF_DL3_HDU_SPECIFICATION["aeff_3d"] = {
        "extname": "EFFECTIVE AREA",
        "column_name": "EFFAREA",
        "mandatory_keywords": {
            **COMMON_IRF_HEADERS,
            "HDUCLAS2": "EFF_AREA",
            "HDUCLAS3": "FULL-ENCLOSURE",  # added here to have HDUCLASN in order
            "HDUCLAS4": "AEFF_3D",
        },
    }

In [156]:
aeff_3d.write("test_aeff3d.fits")

In [157]:
aeff_new = EffectiveArea3D.read("test_aeff3d.fits")
aeff_new

## Energy Dispersion

In [118]:
class EnergyDispersion3D(IRF):
    tag = "edisp_3d"
    required_axes = ["energy_true", "migra", "fov_lon", "fov_lat"]
    default_unit = u.one
    
    @classmethod
    def from_gauss(cls, energy_axis_true, migra_axis, fov_lon_axis, fov_lat_axis, bias, sigma, pdf_threshold=1e-6):
        axes = MapAxes([energy_axis_true, migra_axis, fov_lon_axis, fov_lat_axis])
        coords = axes.get_coord(mode="edges", axis_name="migra")

        migra_min = coords["migra"][:, :-1, :]
        migra_max = coords["migra"][:, 1:, :]

        # Analytical formula for integral of Gaussian
        s = np.sqrt(2) * sigma
        t1 = (migra_max - 1 - bias) / s
        t2 = (migra_min - 1 - bias) / s
        pdf = (scipy.special.erf(t1) - scipy.special.erf(t2)) / 2
        pdf = pdf / (migra_max - migra_min)

        
        r1 = np.rollaxis(pdf, -1, 1)
        r2 = np.rollaxis(r1, 0, -1)
        data = r2 * np.ones(axes.shape)
        
        data[data < pdf_threshold] = 0

        return cls(
            axes=axes,
            data=data.value,
        )

In [125]:
# Make a test case
energy_axis_true = MapAxis.from_energy_bounds(
            "0.1 TeV", "100 TeV", nbin=50, name="energy_true"
        )

migra_axis = MapAxis.from_bounds(
            0, 4, nbin=100, node_type="edges", name="migra"
        )

fov_lon_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lon")
fov_lat_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lat")

energy_true = energy_axis_true.edges[:-1]
sigma = 0.15 / (energy_true / (1 * u.TeV)).value ** 0.3
bias = 1e-3 * (energy_true - 1 * u.TeV).value

edisp3d = EnergyDispersion3D.from_gauss(energy_axis_true=energy_axis_true, 
                                      migra_axis=migra_axis, 
                                      fov_lon_axis=fov_lon_axis, 
                                      fov_lat_axis=fov_lat_axis, 
                                      bias=bias, 
                                      sigma=sigma)
print(edisp3d)

EnergyDispersion3D
------------------

  axes  : ['energy_true', 'migra', 'fov_lon', 'fov_lat']
  shape : (50, 100, 3, 3)
  ndim  : 4
  unit  : 
  dtype : float64



In [126]:
energy = [1, 2] * u.TeV
migra = np.array([0.98, 0.97, 0.7])
fov_lon = [0.1, 1.5] * u.deg
fov_lat = [0.0, 0.3] * u.deg

In [130]:
edisp3d.evaluate(energy_true=energy.reshape(-1, 1, 1, 1), 
               migra=migra.reshape(1, -1, 1, 1), 
               fov_lon=fov_lon.reshape(1, 1, -1, 1), 
               fov_lat=fov_lat.reshape(1, 1, 1, -1))

<Quantity [[[[2.5761652 , 2.5761652 ],
             [2.5761652 , 2.5761652 ]],

            [[2.53395107, 2.53395107],
             [2.53395107, 2.53395107]],

            [[0.38580903, 0.38580903],
             [0.38580903, 0.38580903]]],


           [[[3.14952527, 3.14952527],
             [3.14952527, 3.14952527]],

            [[3.07114116, 3.07114116],
             [3.07114116, 3.07114116]],

            [[0.17595002, 0.17595002],
             [0.17595002, 0.17595002]]]]>

### Serialisation

In [147]:
IRF_DL3_HDU_SPECIFICATION["edisp_3d"] = {
        "extname": "ENERGY DISPERSION",
        "column_name": "MATRIX",
        "mandatory_keywords": {
            **COMMON_IRF_HEADERS,
            "HDUCLAS2": "EDISP",
            "HDUCLAS3": "FULL-ENCLOSURE",  # added here to have HDUCLASN in order
            "HDUCLAS4": "EDISP_3D",
        },
}

In [150]:
edisp3d.write("test_edisp.fits", overwrite=True)

In [153]:
edisp_new = EnergyDispersion3D.read("test_edisp.fits")
edisp_new

## PSF

In [158]:
class PSFnD(IRF):
    tag = "edisp_3d"
    required_axes = ["energy_true", "fov_lon", "fov_lat", "rad"]
    default_unit = u.sr**-1
    
    @classmethod
    def from_parametric_psf(psf_para):
        

In [ ]:
energy_axis = MapAxis.from_energy_edges([0.1, 0.3, 1.0, 3.0, 10.0]*u.TeV, name="energy_true")

nbin=7
fov_lon_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lon")
fov_lat_axis = MapAxis.from_edges([-1.5, -0.5, 0.5, 1.5]*u.deg,  name="fov_lat")

rad_axis = MapAxis.from_edges([0, 1, 2], unit="deg", name="rad")

data = np.ones((4, 3, 3))
for i in range(1,4):
    data[i] = data[i-1]*1.5